# Composite-Adv Demonstration
![CAA Flow](figures/CAA_Flow.png)
This notebook provides a step-by-step demonstration showing how to launch our composite adversarial attack (CAA). We use the CIFAR-10 dataset for demonstration, while other datasets could be executed similarly.

## I. Parameters Configuration

In [1]:
PARAMS = {
    'seed': 0,
    'dataset': {
        'name': 'cifar10',
        'root': '../data/',
        'batch_size': 128,
        'normalize': {
            # CIFAR10
            'mean': [0.4914, 0.4822, 0.4465],
            'std': [0.2023, 0.1994, 0.2010],
            # ImageNet
            # 'mean': [0.485, 0.456, 0.406],
            # 'std': [0.229, 0.224, 0.225],
        }
    },
    'model':{ 
        'madry':{ # Madry's Robust Model
            'arch': 'resnet50',
            'normalize': True,
            'checkpoint': '../GAT/pretrained_model/cifar_madry_resnet50_linf_8.pt', # download from https://github.com/MadryLab/robustness
        },
        'trades':{ # Trades' Robust Model
            'arch': 'wideresnet',
            'normalize': False,
            'checkpoint': '../GAT/pretrained_model/cifar_trades_wideresnet34_10_linf_8.pt', # download from https://github.com/yaodongyu/TRADES
        },
    },
}

In [2]:
import torch, numpy, random
torch.manual_seed(PARAMS['seed'])
numpy.random.seed(PARAMS['seed'])
random.seed(PARAMS['seed'])
torch.backends.cudnn.deterministic = True

## II. Dataset

In [3]:
from composite_adv.utilities import make_dataloader
# load dataset
data_loader = make_dataloader(PARAMS['dataset']['root'], PARAMS['dataset']['name'], transform=None, batch_size=PARAMS['dataset']['batch_size'], shuffle=True)

Files already downloaded and verified


## III. Select Models

### 1. Madry's $\ell_\infty$-robust model

In [4]:
from composite_adv.utilities import make_madry_model, EvalModel
# load model
base_model = make_madry_model(PARAMS['model']['madry']['arch'],
                              PARAMS['dataset']['name'],
                              checkpoint_path=PARAMS['model']['madry']['checkpoint'])

# Normalizer
model = EvalModel(base_model,
                  normalize_param=PARAMS['dataset']['normalize'],
                  input_normalized=PARAMS['model']['madry']['normalize'])

# Send to GPU
import torch
if not torch.cuda.is_available():
    print('using CPU, this will be slow')
else:
    model.cuda()

=> loaded checkpoint '../GAT/pretrained_model/cifar_madry_resnet50_linf_8.pt' (epoch 153)
Natural accuracy --> 87.02999877929688
Robust accuracy --> 55.029998779296875


### 2. TRADES $\ell_\infty$-robust model

In [4]:
from composite_adv.utilities import make_trades_model, EvalModel
# load model
base_model = make_trades_model(PARAMS['model']['trades']['arch'],
                               PARAMS['dataset']['name'],
                               checkpoint_path=PARAMS['model']['trades']['checkpoint'])

# Normalizer
model = EvalModel(base_model,
                  normalize_param=PARAMS['dataset']['normalize'],
                  input_normalized=PARAMS['model']['trades']['normalize'])

# Send to GPU
import torch
if not torch.cuda.is_available():
    print('using CPU, this will be slow')
else:
    model.cuda()

=> loaded checkpoint '../GAT/pretrained_model/cifar_trades_wideresnet34_10_linf_8.pt'


## IV. Evaluate Clean Accuracy

In [ ]:
from composite_adv.utilities import robustness_evaluate
from composite_adv.attacks import NoAttack

attack = NoAttack()
robustness_evaluate(model, attack, data_loader)

## V. Evaluate Robust Accuracy

**CAA Configuration**
1. Attacks Pool Selection. For simpilicity, we use the following abbreviations to specify each attack types.
   `0`: Hue, `1`: Saturation, `2`: Rotation, `3`: Brightness, `4`: Contrast, `5`: $\ell_\infty$

2. Attack Ordering Specify. We provide three ordering options ['fixed','random','scheduled']

**Setup**
```python
from composite_adv.attacks import CompositeAttack
CompositeAttack(model, dataset=PARAMS['dataset']['name'], enabled_attack=(0,), order_schedule="fixed")
```

In [ ]:
from composite_adv.attacks import CompositeAttack
from composite_adv.utilities import robustness_evaluate

###  1. Single Attack $\text{CAA}_\text{Hue}$

In [ ]:
composite_attack = CompositeAttack(model, # The model to be attacked
                                   dataset=PARAMS['dataset']['name'],
                                   enabled_attack=(0,), # Abbr: 0: Hue, 1: Saturation, 2: Rotation, 3: Brightness, 4: Contrast, 5: L-infinity
                                   order_schedule="fixed") # Three ordering options ['fixed','random','scheduled']
robustness_evaluate(model, composite_attack, data_loader)

### 2. Three Attacks $\text{CAA}_{3a}$ (Scheduled Order)

In [ ]:
composite_attack = CompositeAttack(model,
                                   dataset=PARAMS['dataset']['name'],
                                   enabled_attack=(0,1,5),
                                   order_schedule="scheduled")
robustness_evaluate(model, composite_attack, data_loader)

### 3. Semantic Attacks (Random Order)

In [ ]:
composite_attack = CompositeAttack(model,
                                   dataset=PARAMS['dataset']['name'],
                                   enabled_attack=(0,1,2,3,4),
                                   order_schedule="random")
robustness_evaluate(model, composite_attack, data_loader)

### 4. Full Attacks (Scheduled Order)

In [ ]:
composite_attack = CompositeAttack(model,
                                   dataset=PARAMS['dataset']['name'],
                                   enabled_attack=(0,1,2,3,4,5),
                                   order_schedule="scheduled")
robustness_evaluate(model, composite_attack, data_loader)